In [12]:
library(deepLearnR)

{
library(MASS)
data(Boston)
X <- Boston[,2:14]
y <- Boston[,1]
TensorFlowDNNRegressor(modelTag="tfdnnr-01", X=X, y=y, steps=5000)
pred <- TensorFlow.regressorEval(modelTag="tfdnnr-01")
mse <- rPython::python.get("mse")
r2 <- rPython::python.get("r2")
}

In [13]:
r2

[1] -1.077636

In [14]:
{
x.vals <- seq(1:100)
y.vals <- 0.3 + 0.5 * x.vals
lm.tf.fit <- deepLearnR::TensorFlow.SystemLinReg(X = x.vals, Y = y.vals,
epochs = 100000, learning.rate = .00005)
lm.tf.fit
}

$slope
[1] 0.5004605

$intercept
[1] 0.2691519